## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [371]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


df = pd.read_csv('chronic_kidney_disease_full.csv')
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [372]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
rbc      248 non-null object
pc       335 non-null object
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
sod      313 non-null float64
pot      312 non-null float64
hemo     348 non-null float64
pcv      329 non-null float64
wbcc     294 non-null float64
rbcc     269 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer: Sodium is usually utilized by kidneys to extrete waste. Potassium, creatinine, and diabetes are related to chronic kidney conditions. 

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [373]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [374]:
df.dropna(axis=0).T

,3,9,11,14,20,22,27,48,58,71,...,390,391,392,393,394,395,396,397,398,399
age,48,53,63,68,61,48,69,73,73,46,...,52,36,57,43,50,55,42,12,17,58
bp,70,90,70,80,80,80,70,70,80,60,...,80,80,80,60,80,80,70,80,60,80
sg,1.005,1.02,1.01,1.01,1.015,1.025,1.01,1.005,1.02,1.01,...,1.025,1.025,1.02,1.025,1.02,1.02,1.025,1.02,1.025,1.025
al,4,2,3,3,2,4,3,0,2,1,...,0,0,0,0,0,0,0,0,0,0
su,0,0,0,2,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rbc,normal,abnormal,abnormal,normal,abnormal,normal,normal,normal,abnormal,normal,...,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal
pc,abnormal,abnormal,abnormal,abnormal,abnormal,abnormal,abnormal,normal,abnormal,normal,...,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal
pcc,present,present,present,present,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,...,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent
ba,notpresent,notpresent,notpresent,present,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,...,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent,notpresent
bgr,117,70,380,157,173,95,264,70,253,163,...,99,85,133,117,137,140,75,100,114,131


In [375]:
# Going to drop Nans from axis not in features, and drop rows with features but still nan
features = ['sod', 'pot', 'su', 'bgr', 'dm', 'sc','class']
df = df[features]
df.isnull().sum()
df.dropna(inplace=True, axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 3 to 399
Data columns (total 7 columns):
sod      258 non-null float64
pot      258 non-null float64
su       258 non-null float64
bgr      258 non-null float64
dm       258 non-null object
sc       258 non-null float64
class    258 non-null object
dtypes: float64(5), object(2)
memory usage: 16.1+ KB


Answer: We would only have 158 out of the original 400 rows. One downside is we lose too many sample values. The other issue is that having only complete test cases might give us a biased sample set. It may be possible that only 1 hospital filled out the data and it represents ONLY that hospital which may have some laden bias underneath. 

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: Doctors/Nurses are notorious for bad handwriting. Perhaps the data is complete but illegible. IF their diagnosis is incorrect it could also prove cause issues. Perhaps they don't have Chronic Kidney disease and perhaps KD is a symptom of an underlying condition like dehydration which would give unsual data values instead of the disease trying to be diagnosed. Lastly, readings can be impercise such as systolic vs diastolic blood pressure

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: tell everyone they have CKD. 

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Zeroing any false negatives. 

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: It could cause a lot of people to go through trouble, emotional pain, stress, uncessary testing and waste a lot of resources.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple) model can I create that will accomplish this?

Answer: Tell everyone they do not have CKD

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: In this case it is zeroing any false positives. 

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: If someone really has the disease it can a lot of problems if they aren't diagnosed properly

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [376]:
df = pd.get_dummies(df)
df.head()

,sod,pot,su,bgr,sc,dm_no,dm_yes,class_ckd,class_notckd
3,111.0,2.5,0.0,117.0,3.8,1,0,1,0
5,142.0,3.2,0.0,74.0,1.1,0,1,1,0
6,104.0,4.0,0.0,100.0,24.0,1,0,1,0
9,114.0,3.7,0.0,70.0,7.2,0,1,1,0
11,131.0,4.2,0.0,380.0,2.7,0,1,1,0


In [377]:
features = ['sod', 'pot', 'su', 'bgr', 'dm_yes', 'sc']
X = df[features]
y = df['class_ckd']

In [378]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [379]:
ss =StandardScaler()
ss.fit(X_train)
ss.transform(X_train)
ss.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


array([[ 9.40124808e-01, -3.46629377e-01, -4.06323732e-01,
        -6.45232449e-01, -6.31308740e-01, -4.65809131e-01],
       [-4.42833179e-01, -2.19865815e-01, -4.06323732e-01,
        -6.32085293e-01, -6.31308740e-01, -5.46293609e-01],
       [-4.42833179e-01, -2.19865815e-01, -4.06323732e-01,
        -1.85081985e-01, -6.31308740e-01, -5.73121768e-01],
       [-7.19424777e-01,  1.28733980e-01, -4.06323732e-01,
        -1.32493361e-01,  1.58401102e+00,  1.70977389e-02],
       [ 2.48645814e-01, -1.56484034e-01, -4.06323732e-01,
        -1.45640517e-01, -6.31308740e-01, -5.19465449e-01],
       [-4.42833179e-01,  1.92115761e-01, -4.06323732e-01,
         4.06540040e-01,  1.58401102e+00,  3.12207493e-01],
       [ 2.48645814e-01,  1.28733980e-01, -4.06323732e-01,
        -2.24523454e-01, -6.31308740e-01, -5.73121768e-01],
       [-2.79457832e-02, -2.51556706e-01, -4.06323732e-01,
        -3.29700702e-01, -6.31308740e-01, -1.43871218e-01],
       [ 2.48645814e-01, -3.14938487e-01, -4.063

In [380]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
logit.score(X_train, y_train), logit.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.9430051813471503, 0.9384615384615385)

In [381]:
logit.coef_
features = ['sod', 'pot', 'su', 'bgr', 'dm_yes', 'sc']
pd.DataFrame(logit.coef_, columns= features)

,sod,pot,su,bgr,dm_yes,sc
0,-0.038135,-0.242512,0.515887,0.013386,2.495946,2.810228


In [382]:
predict = logit.predict_proba(X_test)

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [383]:
# diabetes had one of the highest relations to having Chronic Kidney Disease. 

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [384]:
#DM Yes (meaning having Diabetes Milletus) was negatively correlated to not having CKD. 
#Another way is that Diabetes has a positive correlation to having CKD

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer: Logistic regression returns useable co-efficents and a more interpretable model.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [385]:
# Test Data. Modeliing because we want to see if our training data is giving us a proper conclusion
predictions = logit.predict(X_test)
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, columns=['predict neg', 'predict pos'], index = ['actual neg', 'actual pos'])
cm_df

,predict neg,predict pos
actual neg,36,0
actual pos,4,25


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: We would want to optimize sensitiivty. Not being diagnosed with a chronic disease, espeically one which is chronic, would result in death, lawsuits, and hosts of other problems. We want to be able to correctly diagnose so we want to optimize our sensivity rate. 

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

We may have a hidden unabalanced class within something like Diabetus Milletus where there are 72 people with Diabetes vs 186 where we do not. Diabetus also is rather impactful to our model. Because of the imbalanced classes we may need to get more data and samples

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: I would take an extreme example. Say something like if we had 1 person with diabetes and that 1 person had Chronic Kidney Disease. Well our data is going to say that Diabetes is 100% correlated to Chronic Kidney Disease. Well that has to do more with sample size than imbalanced classes but image if I had 10 people with Diabetes and 990 who did not. It may look like 5 of the people with Diabetes gives us a rate of 50% but what if of the 1000 that 590 people had CKD. So in actuality an unbalanced class is giving us a very different ratio of explanation than large populations.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 4,600 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with 5,000 observations, of which only about 150 are non-CKD individuals.

In [386]:
df = pd.read_csv('chronic_kidney_disease_full.csv')
choices = df[df['class'] == 'ckd']
appendthis = choices.sample(n=4600, replace=True, random_state=24)

In [387]:
appendthis.shape
newpd = pd.concat([df,appendthis], axis=0)
newpd.shape
features = ['sod', 'pot', 'su', 'bgr', 'dm', 'sc','class']
newpd = newpd[features]
newpd.dropna(inplace=True, axis=0)
newpd.shape

(2520, 7)

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?

In [388]:
newpd.isnull().sum()

sod      0
pot      0
su       0
bgr      0
dm       0
sc       0
class    0
dtype: int64

In [389]:
newpd = pd.get_dummies(newpd)

In [390]:
features = ['sod', 'pot', 'su', 'bgr', 'dm_yes', 'sc']
X = newpd[features]
y = newpd['class_ckd']

In [391]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [392]:
ss =StandardScaler()
ss.fit(X_train)
ss.transform(X_train)
ss.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


array([[-0.14201803, -0.27641103, -0.58991159, -0.50945913,  0.92650261,
        -0.67151386],
       [ 0.67383024, -0.08001372,  0.89964472, -0.12337284,  0.92650261,
         0.62913564],
       [-0.54994217, -0.2567713 , -0.58991159, -0.86337156, -1.07932778,
        -0.65053565],
       ...,
       [ 0.53785553, -0.09965345,  2.38920103,  1.31372612,  0.92650261,
        -0.29390594],
       [-0.41396746, -0.23713157, -0.58991159, -0.89554542,  0.92650261,
        -0.41977525],
       [-0.68591688, -0.49244807,  0.89964472,  1.44242155, -1.07932778,
        -0.27292772]])

In [393]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
logit.score(X_train, y_train), logit.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.9661375661375662, 0.9634920634920635)

In [394]:
logit.coef_
features = ['sod', 'pot', 'su', 'bgr', 'dm_yes', 'sc']
pd.DataFrame(logit.coef_, columns= features)

,sod,pot,su,bgr,dm_yes,sc
0,-0.054458,0.24967,0.266959,0.022732,3.393282,3.477193


In [395]:
predict = logit.predict_proba(X_test)

In [396]:
# The cells which reprsented the supposed unabalnced class have a much higher
# reprsentation within the dataset. Part of it we can see that it proves 
# Our alt hypothesis about diabetes stronger. This is not good as it's misleading data

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!

In [ ]:
# This model has a 92% accuracy rate with 8% rate of false positives. This model
# has much that could be improved upon especially in regards to increasing the
# sensitivity rate. 